## Add control values

In [1]:
if '../' not in sys.path:
    sys.path.append('../')
from src.data.parse_data import *
from src.visualization.advanced_visualizations import *

In [2]:
df_list = list()
for i in range(1,8):
    df_list.append((parse_data(f'Assay {i}.xlsx'), f'Assay {i}.xlsx'))

Assay 1.xlsx - deleted 15 rows with invalid Transfer Status
Assay 2.xlsx - deleted 4 rows with invalid Transfer Status
Assay 3.xlsx - deleted 2 rows with invalid Transfer Status


In [3]:
combined = combine_assays(df_list)
# combined.head()

In [4]:
df_prepared = get_activation_inhibition(combined)
# df_prepared.head()

In [5]:
res = add_control_rows(df_prepared)
# res.tail()

In [6]:
projections = get_projections(res)
# projections.tail()

In [7]:
compounds, ctrl = split_compounds_controls(projections)
# display(compounds.head())
# display(ctrl.head(2))

In [11]:
fig = plot_projection_2d(compounds, '% ACTIVATION - Assay 2')
controls = split_controls_pos_neg(ctrl, '% ACTIVATION - Assay 2')
fig_controls = projection_2d_add_controls(fig, controls)
fig_controls.show()
fig.show()